## Model Conversion Guide

In this notebook, we will consider the full path (*torch* → *onnx* → *tf-protobuf* → *tf-saved_model* → *tf-serving*) of converting models for output to production using the wsl-encoder as an example.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.utils.model_zoo as model_zoo
import numpy as np
from PIL import Image
import torchvision as tv

model_wsl = torch.hub.load('facebookresearch/WSL-Images', 'resnext101_32x8d_wsl')

### Torch

In [2]:
# Encoder model that receives features from the input image
class Wsl_encoder(nn.Module):
    def __init__(self):
        super(Wsl_encoder, self).__init__()
        self.net = model_wsl
        self.net = nn.Sequential(*list(self.net.children())[:-2])

    def forward(self, x):
        x = self.net(x)
        x = x.permute(0, 2, 3, 1)
        x = x.view(x.size(0), -1, x.size(-1))
        return x

torch_encoder = Wsl_encoder()
torch_encoder.eval();

In [3]:
# Input image preprocessing
data_transforms = tv.transforms.Compose([
    tv.transforms.Resize((299, 299)),
    tv.transforms.ToTensor(),
    tv.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def preprocessing_image(img_path):
    with open(img_path, 'rb') as f:
        img = Image.open(f)
        img = img.convert('RGB')
        img = data_transforms(img)
        img = torch.FloatTensor(img)
        img = img.unsqueeze(0)
    return img

In [4]:
# We will get features for two pictures in order to later understand that the model produces different results for different inputs
img_path = r'E:\0.jpg'
img_path_1 = r'E:\1.jpg'
image = preprocessing_image(img_path)
image_1 = preprocessing_image(img_path_1)

torch_features = torch_encoder(image)
torch_features_1 = torch_encoder(image_1)

# Let's look at the output tensor itself
torch_features  # torch.Size([1, 100, 2048])

tensor([[[0.0000, 0.2136, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0406, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 2.0856,  ..., 0.0000, 0.3631, 0.0000],
         [0.0000, 0.0000, 1.6003,  ..., 0.0000, 0.9627, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],
       grad_fn=<ViewBackward>)

### Onnx

In [24]:
import torch.onnx
import onnx
import onnxruntime

In [7]:
# Let's convert the model to onnx
torch.onnx.export(torch_encoder,
                  args=image,
                  f='wsl_encoder.onnx',
                  verbose=True,
                  export_params=True,
                  do_constant_folding=True,
                  opset_version=10,
                  input_names=['input_image'],
                  output_names=['output_features'],
                  # specify that the batch_size parameter can change
                  dynamic_axes={'input_image' : {0 : 'batch_size'},
                                'output_features' : {0 : 'batch_size'}})

graph(%input_image : Float(1, 3, 299, 299),
      %net.0.weight : Float(64, 3, 7, 7),
      %net.1.weight : Float(64),
      %net.1.bias : Float(64),
      %net.1.running_mean : Float(64),
      %net.1.running_var : Float(64),
      %net.4.0.conv1.weight : Float(256, 64, 1, 1),
      %net.4.0.bn1.weight : Float(256),
      %net.4.0.bn1.bias : Float(256),
      %net.4.0.bn1.running_mean : Float(256),
      %net.4.0.bn1.running_var : Float(256),
      %net.4.0.conv2.weight : Float(256, 8, 3, 3),
      %net.4.0.bn2.weight : Float(256),
      %net.4.0.bn2.bias : Float(256),
      %net.4.0.bn2.running_mean : Float(256),
      %net.4.0.bn2.running_var : Float(256),
      %net.4.0.conv3.weight : Float(256, 256, 1, 1),
      %net.4.0.bn3.weight : Float(256),
      %net.4.0.bn3.bias : Float(256),
      %net.4.0.bn3.running_mean : Float(256),
      %net.4.0.bn3.running_var : Float(256),
      %net.4.0.downsample.0.weight : Float(256, 64, 1, 1),
      %net.4.0.downsample.1.weight : Float(256),
  

In [ ]:
# Load the resulting model for test
onnx_model = onnx.load("wsl_encoder.onnx")
onnx.checker.check_model(onnx_model)

ort_session = onnxruntime.InferenceSession("wsl_encoder.onnx")

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# Get output from onnx model
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(image)}
ort_outs = ort_session.run(None, ort_inputs)

# Compare the outputs between torch and onnx models
np.testing.assert_allclose(to_numpy(torch_features), ort_outs[0], rtol=1e-03, atol=1e-04)

print("Conversion successful!")

In [19]:
# Just in case, make sure the output tensor looks the same :)
torch.from_numpy(ort_outs[0])

tensor([[[0.0000, 0.2136, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0406, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 2.0856,  ..., 0.0000, 0.3631, 0.0000],
         [0.0000, 0.0000, 1.6003,  ..., 0.0000, 0.9627, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]])

In [ ]:
# Now let's also check that the resulting model works correctly not only on the input that we used to get it
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(image_1)}
ort_outs = ort_session.run(None, ort_inputs)

np.testing.assert_allclose(to_numpy(torch_features_1), ort_outs[0], rtol=1e-03, atol=1e-04)

print("Conversion successful!")

### Onnx-tf

Now we need to get *tensorflow* graph from our onnx model

In [27]:
from onnx_tf.backend import prepare

In [28]:
# Load model
model = onnx.load('wsl_encoder.onnx')

# Let's convert it to Tensorflow
tf_rep = prepare(model)

# Check inputs and outputs
print('inputs:', tf_rep.inputs)
print('outputs:', tf_rep.outputs)

# Print all internal operations
print('tensor_dict:', tf_rep.tensor_dict)

In [29]:
img_path = '0.jpg'
image = preprocessing_image(img_path)

# Let's run the same picture through the resulting graph
output = tf_rep.run(image)
# Let's see that we got the same tensor as in onnx
print("output:", output)
# Save the graph
tf_rep.export_graph('wsl_onnx_tf.pb')

### Tf-protobuff

Now let's check the work of the resulting graph in *tensorflow*

In [2]:
# Reload the kernel to avoid errors - now we will work only with tensorflow
import tensorflow as tf

tf_encoder = "wsl_onnx_tf.pb"

# Specify the names of the input and output tensors
INPUT_TENSOR_NAME = 'input_image: 0'
OUTPUT_TENSOR_NAME = 'output_features: 0'

# Load the graph
with tf.io.gfile.GFile(tf_encoder,'rb') as encoder_pb:
    graph_enc = tf.compat.v1.GraphDef()
    graph_enc.ParseFromString(encoder_pb.read())
with tf.Graph().as_default() as encoder_graph:
    tf.import_graph_def(graph_enc, name="")

input_tensor = encoder_graph.get_tensor_by_name(INPUT_TENSOR_NAME)
output_tensor = encoder_graph.get_tensor_by_name(OUTPUT_TENSOR_NAME)

# Let's start a session that puts the result of passing through the graph of the input tensor into the specified output tensor
with tf.compat.v1.Session(graph=encoder_graph) as enc_sess:
    encoder_output = enc_sess.run(output_tensor, feed_dict={input_tensor: image.numpy()})  
encoder_output

In [2]:
# You can look at the graph itself
for op in encoder_graph.get_operations():
    print(op.values())

(<tf.Tensor 'Const:0' shape=(1,) dtype=int64>,)
(<tf.Tensor 'Const_1:0' shape=(64, 3, 7, 7) dtype=float32>,)
(<tf.Tensor 'Const_2:0' shape=(64,) dtype=float32>,)
(<tf.Tensor 'Const_3:0' shape=(64,) dtype=float32>,)
(<tf.Tensor 'Const_4:0' shape=(64,) dtype=float32>,)
(<tf.Tensor 'Const_5:0' shape=(64,) dtype=float32>,)
(<tf.Tensor 'Const_6:0' shape=(256,) dtype=float32>,)
(<tf.Tensor 'Const_7:0' shape=(256,) dtype=float32>,)
(<tf.Tensor 'Const_8:0' shape=(256,) dtype=float32>,)
(<tf.Tensor 'Const_9:0' shape=(256,) dtype=float32>,)
(<tf.Tensor 'Const_10:0' shape=(256,) dtype=float32>,)
(<tf.Tensor 'Const_11:0' shape=(256,) dtype=float32>,)
(<tf.Tensor 'Const_12:0' shape=(256,) dtype=float32>,)
(<tf.Tensor 'Const_13:0' shape=(256,) dtype=float32>,)
(<tf.Tensor 'Const_14:0' shape=(256,) dtype=float32>,)
(<tf.Tensor 'Const_15:0' shape=(256,) dtype=float32>,)
(<tf.Tensor 'Const_16:0' shape=(256,) dtype=float32>,)
(<tf.Tensor 'Const_17:0' shape=(256,) dtype=float32>,)
(<tf.Tensor 'Const_18:0

(<tf.Tensor 'split_106:0' shape=(3, 3, 32, 32) dtype=float32>, <tf.Tensor 'split_106:1' shape=(3, 3, 32, 32) dtype=float32>, <tf.Tensor 'split_106:2' shape=(3, 3, 32, 32) dtype=float32>, <tf.Tensor 'split_106:3' shape=(3, 3, 32, 32) dtype=float32>, <tf.Tensor 'split_106:4' shape=(3, 3, 32, 32) dtype=float32>, <tf.Tensor 'split_106:5' shape=(3, 3, 32, 32) dtype=float32>, <tf.Tensor 'split_106:6' shape=(3, 3, 32, 32) dtype=float32>, <tf.Tensor 'split_106:7' shape=(3, 3, 32, 32) dtype=float32>, <tf.Tensor 'split_106:8' shape=(3, 3, 32, 32) dtype=float32>, <tf.Tensor 'split_106:9' shape=(3, 3, 32, 32) dtype=float32>, <tf.Tensor 'split_106:10' shape=(3, 3, 32, 32) dtype=float32>, <tf.Tensor 'split_106:11' shape=(3, 3, 32, 32) dtype=float32>, <tf.Tensor 'split_106:12' shape=(3, 3, 32, 32) dtype=float32>, <tf.Tensor 'split_106:13' shape=(3, 3, 32, 32) dtype=float32>, <tf.Tensor 'split_106:14' shape=(3, 3, 32, 32) dtype=float32>, <tf.Tensor 'split_106:15' shape=(3, 3, 32, 32) dtype=float32>, <

### Tensorflow Graph Transform

### Tf-saved_model

Now we will prepare the resulting model for serving by translating it into the appropriate special format *saved_model*

In [3]:
from tensorflow.python.saved_model import utils as smutils
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import signature_def_utils
from tensorflow.python.saved_model import tag_constants

In [7]:
optimized_encoder = "optimized_wsl.pb" 

# Repeat the above operations
with tf.io.gfile.GFile(optimized_encoder,'rb') as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())
with tf.Graph().as_default() as graph:
    tf.import_graph_def(graph_def, name="")

with tf.compat.v1.Session(graph=graph) as sess:
    export_dir = 'saved_encoder'

    input_tensor = graph.get_tensor_by_name('input_image: 0')  
    output_tensor = graph.get_tensor_by_name('output_features: 0')

    # Set the names of inputs and outputs for serving
    tensor_info_inputs = {'images': smutils.build_tensor_info(input_tensor)}

    tensor_info_outputs = {'extracted_features': smutils.build_tensor_info(output_tensor)} 

    prediction_signature = signature_def_utils.build_signature_def(
                            inputs=tensor_info_inputs,
                            outputs=tensor_info_outputs,
                            method_name=signature_constants.PREDICT_METHOD_NAME)

    builder = tf.compat.v1.saved_model.builder.SavedModelBuilder(export_dir)
    builder.add_meta_graph_and_variables(
        sess, [tag_constants.SERVING],
        signature_def_map={signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: prediction_signature})
    builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: saved_encoder\saved_model.pb


### Tf-serving

Now let's see how to start serving our model on the server.

In [ ]:
import grpc
import numpy as np
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc
from PIL import Image

In [ ]:
# Increase the message lengths of the request and create a channel for it
options = [('grpc.max_send_message_length', 512 * 1024 * 1024), ('grpc.max_receive_message_length', 512 * 1024 * 1024)]
channel = grpc.insecure_channel('0.0.0.0:9000', options = options)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

# Specify the same model name that we specified in the commands above
request_encoder = predict_pb2.PredictRequest()
request_encoder.model_spec.name = 'encoder'

In [ ]:
image = 'here is our preprocessed image converted to numpy array'

# Send an image to the model, at the output we get a numpy array of features
request_encoder.inputs['images'].CopyFrom(tf.make_tensor_proto(image))
encoder_out = tf.make_ndarray(stub.Predict(request_encoder).outputs['extracted_features'])      

Done - the model is spinning :)